In [1]:
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf
from plotly.subplots import make_subplots
from dash import Dash, dcc, html
import pandas_market_calendars as mcal

In [2]:
pd.set_option("display.max_rows", 15_000)
pd.set_option("display.max_columns", 500)

In [3]:
import sys
sys.path.append('..')
from tooling.filter import find_atm, option_tool
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm
from tooling.enums import Index, AssetClass, StrikeSpread, Spot

In [4]:
def calculate_atr(df, length, ma_20_length, ma_60_length):
    atr_values = [0] * len(df)
    for i in range(1, len(df)):
        tr = max(df.iloc[i]["h"] - df.iloc[i]["l"], abs(df.iloc[i]["h"] - df.iloc[i - 1]["c"]), abs(df.iloc[i]["l"] - df.iloc[i - 1]["c"]))
        atr_values[i] = int(((length - 1) * atr_values[i-1] + tr) / length)  # Convert to int here if necessary
    df["ATR"] = atr_values
    
    # Calculate Moving Averages
    df["ATR_MA20"] = df["ATR"].rolling(window=ma_20_length).mean().fillna(0).astype(int)
    df["ATR_MA60"] = df["ATR"].rolling(window=ma_60_length).mean().fillna(0).astype(int)
    df["Moving Average"] = df['c'].rolling(window=14).mean().fillna(0).astype(int)
    
    # Generate Signals
    df["signal"] = 0  # Initialize signal column with zeros
    df.loc[(df["ATR"].shift(1) < df["ATR_MA20"].shift(1)) & (df["ATR"] >= df["ATR_MA20"]) & (df["ATR_MA20"] < df["ATR_MA60"]), "signal"] = 1
    
    # Convert specific columns to integer if desired
    # df[["ATR", "ATR_MA20", "ATR_MA60", "Moving Average", "signal"]] = df[["ATR", "ATR_MA20", "ATR_MA60", "Moving Average", "signal"]].astype(int)
    
    # Or convert the entire DataFrame to integers, if all columns should be integers
    # df = df.astype(int)
    
    return df


In [7]:
import os
path = os.getcwd()
path

'/home/jagjot/strategies/droplets/droplets/Attar'

In [8]:
# dataframe_crude_oil = pd.read_csv('../CrudeOilData/crude_oil_1min.csv')
# dataframe_crude_oil = pd.read_csv('../CrudeOilData/crude_oil_data_hourly.csv')
dataframe_crude_oil = pd.read_csv('../CrudeOil_1Min_New.csv')
dataframe_crude_oil

,Unnamed: 0,datetime,o,h,l,c
0,0,2017-01-02 10:01:00,3666.0,3668.0,3663.0,3665.0
1,1,2017-01-02 10:02:00,3665.0,3673.0,3663.0,3672.0
2,2,2017-01-02 10:03:00,3672.0,3673.0,3669.0,3670.0
3,3,2017-01-02 10:04:00,3670.0,3671.0,3669.0,3670.0
4,4,2017-01-02 10:05:00,3670.0,3670.0,3667.0,3667.0
...,...,...,...,...,...,...
1615251,1615251,2023-12-29 23:55:00,5992.0,5999.0,5992.0,5999.0
1615252,1615252,2023-12-29 23:56:00,5992.0,5999.0,5992.0,5999.0
1615253,1615253,2023-12-29 23:57:00,5992.0,5999.0,5992.0,5999.0
1615254,1615254,2023-12-29 23:58:00,5992.0,5999.0,5992.0,5999.0


In [9]:
# bnf = pd.read_csv('data/bnf_min.csv')
bnf = dataframe_crude_oil

In [10]:
bnf['datetime'] = pd.to_datetime(bnf['datetime'])
bnf = bnf.drop(columns=['Unnamed: 0'])
bnf

,datetime,o,h,l,c
0,2017-01-02 10:01:00,3666.0,3668.0,3663.0,3665.0
1,2017-01-02 10:02:00,3665.0,3673.0,3663.0,3672.0
2,2017-01-02 10:03:00,3672.0,3673.0,3669.0,3670.0
3,2017-01-02 10:04:00,3670.0,3671.0,3669.0,3670.0
4,2017-01-02 10:05:00,3670.0,3670.0,3667.0,3667.0
...,...,...,...,...,...
1615251,2023-12-29 23:55:00,5992.0,5999.0,5992.0,5999.0
1615252,2023-12-29 23:56:00,5992.0,5999.0,5992.0,5999.0
1615253,2023-12-29 23:57:00,5992.0,5999.0,5992.0,5999.0
1615254,2023-12-29 23:58:00,5992.0,5999.0,5992.0,5999.0


In [11]:
dataframe_crude_oil.reset_index(drop=True, inplace=True)
# dataframe_crude_oil['Datetime'] = pd.to_datetime(dataframe_crude_oil['datetime'])
dataframe_crude_oil.set_index('datetime', inplace=True)
# dataframe_crude_oil['datetime'] = pd.to_datetime(dataframe_crude_oil['datetime'])
dataframe_crude_oil

,Unnamed: 0,o,h,l,c
datetime,,,,,
2017-01-02 10:01:00,0,3666.0,3668.0,3663.0,3665.0
2017-01-02 10:02:00,1,3665.0,3673.0,3663.0,3672.0
2017-01-02 10:03:00,2,3672.0,3673.0,3669.0,3670.0
2017-01-02 10:04:00,3,3670.0,3671.0,3669.0,3670.0
2017-01-02 10:05:00,4,3670.0,3670.0,3667.0,3667.0
...,...,...,...,...,...
2023-12-29 23:55:00,1615251,5992.0,5999.0,5992.0,5999.0
2023-12-29 23:56:00,1615252,5992.0,5999.0,5992.0,5999.0
2023-12-29 23:57:00,1615253,5992.0,5999.0,5992.0,5999.0


In [12]:
import pandas as pd

# Set the timestamp column as the index
# df = bnf
df = dataframe_crude_oil
# df.set_index('datetime', inplace=True)

# Resample to 15-minute OHLC data
ohlc_15min = df.resample('30min').agg({
    'o': 'first',
    'h': 'max',
    'l': 'min',
    'c': 'last',
})

# Drop any rows with NaN values
ohlc_15min.dropna(inplace=True)

# Reset index if you want timestamp as a column instead of the index
# ohlc_15min.reset_index(inplace=True)

crude_15min = ohlc_15min
crude_15min

,o,h,l,c
datetime,,,,
2017-01-02 10:00:00,3666.0,3673.0,3663.0,3672.0
2017-01-02 10:30:00,3672.0,3675.0,3670.0,3672.0
2017-01-02 11:00:00,3672.0,3673.0,3671.0,3673.0
2017-01-02 11:30:00,3673.0,3675.0,3672.0,3673.0
2017-01-02 12:00:00,3673.0,3674.0,3671.0,3672.0
...,...,...,...,...
2023-12-29 21:30:00,5988.0,6036.0,5986.0,6027.0
2023-12-29 22:00:00,6028.0,6033.0,5979.0,5982.0
2023-12-29 22:30:00,5982.0,6027.0,5982.0,6025.0


In [13]:
# crude_oil_df = dataframe_crude_oil
# # crude_oil_df = pd.read_csv('crude_oil_data_hourly.csv')
# dataframe_crude_oil = dataframe_crude_oil.rename(columns={'open': 'o', 'high': 'h', 'low': 'l', 'close': 'c', 'time': 'datetime'})
# crude_oil_df.drop(columns=['Volume'], inplace=True)
# crude_oil_df
# bnf_5min
dataframe_crude_oil = crude_15min
dataframe_crude_oil

,o,h,l,c
datetime,,,,
2017-01-02 10:00:00,3666.0,3673.0,3663.0,3672.0
2017-01-02 10:30:00,3672.0,3675.0,3670.0,3672.0
2017-01-02 11:00:00,3672.0,3673.0,3671.0,3673.0
2017-01-02 11:30:00,3673.0,3675.0,3672.0,3673.0
2017-01-02 12:00:00,3673.0,3674.0,3671.0,3672.0
...,...,...,...,...
2023-12-29 21:30:00,5988.0,6036.0,5986.0,6027.0
2023-12-29 22:00:00,6028.0,6033.0,5979.0,5982.0
2023-12-29 22:30:00,5982.0,6027.0,5982.0,6025.0


In [1018]:
dataframe_crude_oil = dataframe_crude_oil.rename(columns={'open': 'o', 'high': 'h', 'low': 'l', 'close': 'c'})

In [14]:
# df = crude_oil_df
df = dataframe_crude_oil
# dataframe_crude_oil.set_index('datetime', inplace=True)
df['datetime'] = df.index
df.dropna(inplace=True)
df

,o,h,l,c,datetime
datetime,,,,,
2017-01-02 10:00:00,3666.0,3673.0,3663.0,3672.0,2017-01-02 10:00:00
2017-01-02 10:30:00,3672.0,3675.0,3670.0,3672.0,2017-01-02 10:30:00
2017-01-02 11:00:00,3672.0,3673.0,3671.0,3673.0,2017-01-02 11:00:00
2017-01-02 11:30:00,3673.0,3675.0,3672.0,3673.0,2017-01-02 11:30:00
2017-01-02 12:00:00,3673.0,3674.0,3671.0,3672.0,2017-01-02 12:00:00
...,...,...,...,...,...
2023-12-29 21:30:00,5988.0,6036.0,5986.0,6027.0,2023-12-29 21:30:00
2023-12-29 22:00:00,6028.0,6033.0,5979.0,5982.0,2023-12-29 22:00:00
2023-12-29 22:30:00,5982.0,6027.0,5982.0,6025.0,2023-12-29 22:30:00


In [501]:
# dataframe_crude_oil2

In [15]:
pd.set_option("display.max_rows", 16_000)
pd.set_option("display.max_columns", 500)

atr_df = calculate_atr(dataframe_crude_oil, 10, 20, 60)
# atr_df['datetime'] = pd.to_datetime(atr_df['datetime'])
# atr_df.drop(atr_df.index[:15000], inplace=True)

In [16]:
atr_df

,o,h,l,c,datetime,ATR,ATR_MA20,ATR_MA60,Moving Average,signal
datetime,,,,,,,,,,
2017-01-02 10:00:00,3666.0,3673.0,3663.0,3672.0,2017-01-02 10:00:00,0,0,0,0,0
2017-01-02 10:30:00,3672.0,3675.0,3670.0,3672.0,2017-01-02 10:30:00,0,0,0,0,0
2017-01-02 11:00:00,3672.0,3673.0,3671.0,3673.0,2017-01-02 11:00:00,0,0,0,0,0
2017-01-02 11:30:00,3673.0,3675.0,3672.0,3673.0,2017-01-02 11:30:00,0,0,0,0,0
2017-01-02 12:00:00,3673.0,3674.0,3671.0,3672.0,2017-01-02 12:00:00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2023-12-29 21:30:00,5988.0,6036.0,5986.0,6027.0,2023-12-29 21:30:00,31,22,23,6018,0
2023-12-29 22:00:00,6028.0,6033.0,5979.0,5982.0,2023-12-29 22:00:00,33,22,23,6014,0
2023-12-29 22:30:00,5982.0,6027.0,5982.0,6025.0,2023-12-29 22:30:00,34,23,23,6016,0


In [1026]:
# # Intraday ...
# import pandas as pd

# def execute_trades(df):
#     entry_high = 0
#     entry_low = 0
#     stoploss_long = 0
#     stoploss_short = 0
#     points_captured = []  # List to store points captured
#     signal_candle_flag = False
#     long_flag = False
#     short_flag = False
#     trade_book = []

#     # Execute Trades
#     for i in range(1, len(df)):
#         points = 0
#         candle_open = df.iloc[i]['o']
#         candle_high = df.iloc[i]['h']
#         candle_low = df.iloc[i]['l']
#         candle_close = df.iloc[i]['c']
#         if df.iloc[i]["signal"] == 1:
#             entry_high = df.iloc[i]["h"]
#             entry_low = df.iloc[i]["l"]
#             signal_candle_flag = True

#         if signal_candle_flag and not (long_flag or short_flag):
#             if entry_high > 0 and df.iloc[i]["h"] > entry_high and not long_flag:
#                 # Long trade
#                 stoploss_long = entry_low
#                 entry_time = df.iloc[i]['datetime']
#                 entry_price = entry_high
#                 long_flag = True
#                 trade_type = 'LONG'
#                 sl_in_points = entry_price - stoploss_long
#                 target_in_points = 6 * sl_in_points
                
#             elif entry_low > 0 and df.iloc[i]["l"] < entry_low and not short_flag:
#                 # Short trade
#                 stoploss_short = entry_high
#                 entry_time = df.iloc[i]['datetime']
#                 entry_price = entry_low
#                 short_flag = True
#                 trade_type = 'SHORT'
#                 sl_in_points = stoploss_short - entry_price
#                 target_in_points = 6 * sl_in_points

#         if long_flag and not points:
#             if (df.iloc[i]["h"] - entry_price) >= target_in_points:
#                 points = target_in_points
#                 signal_candle_flag = False
#                 long_flag = False
#                 exit_price = entry_price + points
#             elif entry_high > 0 and df.iloc[i]["datetime"].hour >= 15 and df.iloc[i]['datetime'].minute == 0:
#                 points = df.iloc[i]["c"] - entry_price
#                 signal_candle_flag = False
#                 long_flag = False
#                 exit_price = df.iloc[i]['c']
#             elif df.iloc[i]["l"] <= stoploss_long:
#                 # points = df.iloc[i]['o'] - entry_price
#                 points = (sl_in_points) * -1
#                 signal_candle_flag = False
#                 long_flag = False
#                 exit_price = stoploss_long

#         if short_flag and not points:
#             if (entry_price - df.iloc[i]["l"]) >= target_in_points:
#                 points = target_in_points
#                 signal_candle_flag = False
#                 short_flag = False
#                 exit_price = entry_price - points
#             elif entry_low > 0 and df.iloc[i]["datetime"].hour >= 15 and df.iloc[i]['datetime'].minute == 0:
#                 points = entry_price - df.iloc[i]["c"]
#                 signal_candle_flag = False
#                 short_flag = False
#                 exit_price = df.iloc[i]['c']
#             elif df.iloc[i]["h"] >= stoploss_short:
#                 # points = entry_price - df.iloc[i]['o']
#                 points = (sl_in_points) * -1
#                 signal_candle_flag = False
#                 short_flag = False
#                 exit_price = stoploss_short
#         if points:
#             # final_points = entry_price - df.iloc[i]["c"] if trade_type == 'SHORT' else df.iloc[i]["c"] - entry_price
#             final_points = points
#             trade_book.append({'Entry Time': entry_time, 'Trade Type': trade_type, 'Entry Price': entry_price,
#                                'Stoploss In Points': entry_price - stoploss_long if trade_type == 'LONG' else stoploss_short - entry_price,
#                                'Exit Time': df.iloc[i]['datetime'], 'Exit Price': exit_price,
#                                'SL or Target': 'EOD' if df.iloc[i]["datetime"].hour >= 15 else ('Target' if points > 0 else 'SL'),
#                                'Points Captured': final_points, 'After Costs': (points-3)})
            
#     # Convert the list of trades into a DataFrame
#     trade_book_df = pd.DataFrame(trade_book)
#     return trade_book_df
    
# # Example usage
# # Assuming `df` is your DataFrame containing OHLC data
# # trade_book = execute_trades(df)
# # print(trade_book)


In [1027]:
# tradebook = execute_trades(atr_df)
# tradebook

In [1028]:
# tradebook.to_csv('Intraday 3R - Attar JJ-SU.csv')

In [1029]:
# pnl = tradebook['After Costs'].sum()
# pnl

In [1030]:
# filtered = tradebook.loc[tradebook['SL or Target'] == 'SL']
# print(len(filtered))

In [1031]:
# filtered

In [1032]:
# profitability = (filtered['After Costs'].sum())/len(filtered)
# profitability

In [33]:
# With Trailing ... 
import pandas as pd

def execute_trades_with_trailing(df):
    entry_high = 0
    entry_low = 0
    stoploss_long = 0
    stoploss_short = 0
    points_captured = []  # List to store points captured
    signal_candle_flag = False
    long_flag = False
    short_flag = False
    trade_book = []

    # Execute Trades
    for i in range(1, len(df)):
        points = 0
        candle_open = df.iloc[i]['o']
        candle_high = df.iloc[i]['h']
        candle_low = df.iloc[i]['l']
        candle_close = df.iloc[i]['c']
        moving_average = df.iloc[i]['Moving Average']  # New column for moving average

        if df.iloc[i]["signal"] == 1:
            entry_high = df.iloc[i]["h"]
            entry_low = df.iloc[i]["l"]
            signal_candle_flag = True
            # initial_sl = 0

        if signal_candle_flag and not (long_flag or short_flag):
            if entry_high > 0 and candle_high > entry_high and not long_flag:
                # Long trade
                stoploss_long = entry_low
                entry_time = df.iloc[i]['datetime']
                entry_price = entry_high
                long_flag = True
                trade_type = 'LONG'
                sl_in_points = entry_price - stoploss_long
                initial_sl = stoploss_long
                target_in_points = 40 * sl_in_points
                
            elif entry_low > 0 and candle_low < entry_low and not short_flag:
                # Short trade
                stoploss_short = entry_high
                entry_time = df.iloc[i]['datetime']
                entry_price = entry_low
                short_flag = True
                trade_type = 'SHORT'
                sl_in_points = stoploss_short - entry_price
                initial_sl = stoploss_short
                target_in_points = 40 * sl_in_points

        if long_flag and not points:
            if (candle_high - entry_price) >= target_in_points:
                points = target_in_points
                signal_candle_flag = False
                long_flag = False
                exit_price = entry_price + points
            elif entry_high > 0 and df.iloc[i]["datetime"].hour >= 15:
                points = candle_close - entry_price
                signal_candle_flag = False
                long_flag = False
                exit_price = candle_close
            elif candle_low > moving_average:
                stoploss_long = max(stoploss_long, moving_average)  # Trail stop-loss
            elif candle_close <= stoploss_long:
                points = candle_close - entry_price
                signal_candle_flag = False
                long_flag = False
                exit_price = candle_close

        if short_flag and not points:
            if (entry_price - candle_low) >= target_in_points:
                points = target_in_points
                signal_candle_flag = False
                short_flag = False
                exit_price = entry_price - points
            elif entry_low > 0 and df.iloc[i]["datetime"].hour >= 15:
                points = entry_price - candle_close
                signal_candle_flag = False
                short_flag = False
                exit_price = candle_close
            elif candle_high < moving_average:
                stoploss_short = min(stoploss_short, moving_average)  # Trail stop-loss
            elif candle_close >= stoploss_short:
                points = entry_price - candle_close
                signal_candle_flag = False
                short_flag = False
                exit_price = candle_close
                
        if points:
            final_points = points
            trade_book.append({'Entry Time': entry_time, 'Trade Type': trade_type, 'Entry Price': entry_price,
                               'Stoploss In Points': entry_price - stoploss_long if trade_type == 'LONG' else stoploss_short - entry_price,
                               'Initial SL': initial_sl, 'Exit Time': df.iloc[i]['datetime'], 'Exit Price': exit_price,
                               'SL or Target': 'Target' if points > 0 else 'SL',
                               'Points Captured': final_points, 'After Costs': (points-3), 'ROI%': (points-3)*15/1000, 'year': entry_time.year})
            
    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df


In [34]:
tradebook_w_trail = execute_trades_with_trailing(atr_df)
tradebook_w_trail

,Entry Time,Trade Type,Entry Price,Stoploss In Points,Initial SL,Exit Time,Exit Price,SL or Target,Points Captured,After Costs,ROI%,year
0,2017-01-05 17:30:00,LONG,3659.0,20.0,3639.0,2017-01-05 17:30:00,3651.0,SL,-8.0,-11.0,-0.165,2017
1,2017-01-06 20:00:00,SHORT,3672.0,28.0,3700.0,2017-01-06 20:00:00,3666.0,Target,6.0,3.0,0.045,2017
2,2017-01-09 16:30:00,LONG,3637.0,10.0,3627.0,2017-01-09 16:30:00,3630.0,SL,-7.0,-10.0,-0.150,2017
3,2017-01-10 18:30:00,LONG,3560.0,8.0,3552.0,2017-01-10 18:30:00,3562.0,Target,2.0,-1.0,-0.015,2017
4,2017-01-11 18:30:00,LONG,3495.0,11.0,3484.0,2017-01-11 18:30:00,3501.0,Target,6.0,3.0,0.045,2017
5,2017-01-13 18:00:00,LONG,3586.0,12.0,3574.0,2017-01-13 18:00:00,3589.0,Target,3.0,0.0,0.000,2017
6,2017-01-16 16:00:00,SHORT,3579.0,16.0,3595.0,2017-01-16 16:00:00,3578.0,Target,1.0,-2.0,-0.030,2017
7,2017-01-16 23:30:00,LONG,3588.0,25.0,3563.0,2017-01-16 23:30:00,3573.0,SL,-15.0,-18.0,-0.270,2017
8,2017-01-19 19:00:00,SHORT,3514.0,10.0,3524.0,2017-01-19 19:00:00,3517.0,SL,-3.0,-6.0,-0.090,2017
9,2017-01-20 20:30:00,LONG,3655.0,16.0,3639.0,2017-01-20 20:30:00,3658.0,Target,3.0,0.0,0.000,2017


In [35]:
tradebook_w_trail.to_csv('Testinggg 6R JJ_SU.csv')

In [36]:
pnl_w_trail = tradebook_w_trail['After Costs'].sum()
pnl_w_trail

1214.0

In [21]:
targets = tradebook_w_trail.loc[tradebook_w_trail['SL or Target'] == 'Target']
# targets2 = targets.loc[targets['Stoploss In Points'] >= 0]
targets

,Entry Time,Trade Type,Entry Price,Stoploss In Points,Initial SL,Exit Time,Exit Price,SL or Target,Points Captured,After Costs,ROI%,year
1,2017-01-06 20:00:00,SHORT,3672.0,28.0,3700.0,2017-01-06 20:00:00,3666.0,Target,6.0,3.0,0.045,2017
3,2017-01-10 18:30:00,LONG,3560.0,8.0,3552.0,2017-01-10 18:30:00,3562.0,Target,2.0,-1.0,-0.015,2017
4,2017-01-11 18:30:00,LONG,3495.0,11.0,3484.0,2017-01-11 18:30:00,3501.0,Target,6.0,3.0,0.045,2017
5,2017-01-13 18:00:00,LONG,3586.0,12.0,3574.0,2017-01-13 18:00:00,3589.0,Target,3.0,0.0,0.000,2017
6,2017-01-16 16:00:00,SHORT,3579.0,16.0,3595.0,2017-01-16 16:00:00,3578.0,Target,1.0,-2.0,-0.030,2017
9,2017-01-20 20:30:00,LONG,3655.0,16.0,3639.0,2017-01-20 20:30:00,3658.0,Target,3.0,0.0,0.000,2017
10,2017-01-23 16:00:00,SHORT,3596.0,17.0,3613.0,2017-01-23 16:00:00,3592.0,Target,4.0,1.0,0.015,2017
13,2017-02-01 18:30:00,LONG,3596.0,9.0,3587.0,2017-02-01 18:30:00,3600.0,Target,4.0,1.0,0.015,2017
15,2017-02-06 19:00:00,SHORT,3608.0,17.0,3625.0,2017-02-06 19:00:00,3606.0,Target,2.0,-1.0,-0.015,2017
17,2017-02-10 17:30:00,LONG,3584.0,43.0,3541.0,2017-02-10 17:30:00,3592.0,Target,8.0,5.0,0.075,2017


In [22]:
# len(targets2)

In [23]:
# tradebook_w_trail.to_csv('Attar 8R new intraday with trailing.csv')

In [24]:
# atr_df

In [25]:
# # Positional ...
# import pandas as pd

# def execute_trades_positional(df):
#     entry_high = 0
#     entry_low = 0
#     stoploss_long = 0
#     stoploss_short = 0
#     sl_in_points = 0
#     points_captured = []  # List to store points captured
#     signal_candle_flag = False
#     long_flag = False
#     short_flag = False
#     trade_book = []

#     # Execute Trades
#     for i in range(1, len(df)):
#         points = 0
#         candle_open = df.iloc[i]['o']
#         candle_high = df.iloc[i]['h']
#         candle_low = df.iloc[i]['l']
#         candle_close = df.iloc[i]['c']
#       # print('Current SL in Points : ', sl_in_points)
#         if df.iloc[i]["signal"] == 1:
#           # print('Signal Candle Generated at : ', df.iloc[i]['datetime'])
#             entry_high = df.iloc[i]["h"]
#             entry_low = df.iloc[i]["l"]
#           # print('Candle High : ', candle_high, ' Candle Low : ', candle_low)
#             signal_candle_flag = True

#         if signal_candle_flag and not (long_flag or short_flag):
#             if entry_high > 0 and df.iloc[i]["h"] > entry_high and not long_flag:
#                 # Long trade
#               # print('Long Trade Triggered !!!')
#                 if df.iloc[i]['o'] >= entry_high:
#                     entry_price = df.iloc[i]['o']
#                 else:
#                     entry_price = entry_high
#               # print('Entry Triggered at Price : ', entry_price)
#                 stoploss_long = entry_low
#               # print('Stoploss at : ', stoploss_long)
#                 entry_time = df.iloc[i]['datetime']
#               # print('Entry Triggered at Time : ', entry_time)
#                 long_flag = True
#                 trade_type = 'LONG'
#                 sl_in_points = entry_price - stoploss_long
#                 target_in_points = 20 * sl_in_points
#               # print('Target Defined At : ', target_in_points + entry_price)
                
#             elif entry_low > 0 and df.iloc[i]["l"] < entry_low and not short_flag:
#                 # Short trade
#               # print('Short Trade Triggered!!!')
#                 if df.iloc[i]['o'] <= entry_low:
#                     entry_price = df.iloc[i]['o']
#                 else:
#                     entry_price = entry_low
#               # print('Entry Triggered at Price : ', entry_price)
#                 stoploss_short = entry_high
#               # print('Stoploss at : ', stoploss_short)
#                 entry_time = df.iloc[i]['datetime']
#               # print('Entry Triggered at Time : ', entry_time)
#                 short_flag = True
#                 trade_type = 'SHORT'
#                 sl_in_points = stoploss_short - entry_price
#                 target_in_points = 20 * sl_in_points
#               # print('Target Defined At : ', entry_price - target_in_points)

#         if long_flag and not points:
#           # print('Inside Long Trade !!!')
#             if (df.iloc[i]["h"] - entry_price) >= target_in_points:
#               # print('Target Achieved at time : ', df.iloc[i]['datetime'])
#                 points = target_in_points
#                 # signal_candle_flag = False
#                 long_flag = False
#                 exit_price = entry_price + points
#               # print('Exit Price : ', exit_price)
            
#             elif df.iloc[i]["datetime"].hour >= 15:
#                 points = df.iloc[i]['c'] - entry_price
#                 # signal_candle_flag = False
#                 long_flag = False
#                 exit_price = entry_price + points
                
#             elif df.iloc[i]["l"] <= stoploss_long:
#               # print('SL hit at time : ', df.iloc[i]['datetime'])
#                 if df.iloc[i]['o'] <= stoploss_long:
#                     points = df.iloc[i]['o'] - entry_price
#                 else:
#                     points = (sl_in_points) * -1
#                 # signal_candle_flag = False
#                 long_flag = False
#                 exit_price = stoploss_long
#               # print('Exit Price : ', exit_price)
                
#                 if df.iloc[i]['datetime'].day != entry_time.day:
#                     points = df.iloc[i]['o'] - entry_price
            
#             elif (df.iloc[i]['ATR'] <= df.iloc[i]['ATR_MA20']) and (df.iloc[i-1]['ATR'] >= df.iloc[i-1]['ATR_MA20']):
#                 points = df.iloc[i]['c'] - entry_price
#                 # signal_candle_flag = False
#                 long_flag = False
#                 exit_price = entry_price + points


#         if short_flag and not points:
#           # print('Inside Short Trade !!!')
#             if (entry_price - df.iloc[i]["l"]) >= target_in_points:
#               # print('Target Achieved at time : ', df.iloc[i]['datetime'])
#                 points = target_in_points
#                 # signal_candle_flag = False
#                 short_flag = False
#                 exit_price = entry_price - points
#               # print('Exit Price : ', exit_price)

#             elif df.iloc[i]["datetime"].hour >= 15:
#                 points = entry_price - df.iloc[i]['c']
#                 # signal_candle_flag = False
#                 short_flag = False
#                 exit_price = entry_price - points
                
#             elif df.iloc[i]["h"] >= stoploss_short:
#               # print('SL hit at time : ', df.iloc[i]['datetime'])
#                 if df.iloc[i]['o'] >= stoploss_short:
#                     points = entry_price - df.iloc[i]['o']
#                 else:
#                     points = (sl_in_points) * -1
#                 # signal_candle_flag = False
#                 short_flag = False
#                 exit_price = stoploss_short
#               # print('Exit Price : ', exit_price)

#                 if df.iloc[i]['datetime'].day != entry_time.day:
#                     points = entry_price - df.iloc[i]['o']
            
#             elif (df.iloc[i]['ATR'] <= df.iloc[i]['ATR_MA20']) and (df.iloc[i-1]['ATR'] >= df.iloc[i-1]['ATR_MA20']):
#                 points = entry_price - df.iloc[i]['c']
#                 # signal_candle_flag = False
#                 short_flag = False
#                 exit_price = entry_price - points

        
#         if points and signal_candle_flag:
#             signal_candle_flag = False
#             # final_points = entry_price - df.iloc[i]["c"] if trade_type == 'SHORT' else df.iloc[i]["c"] - entry_price
#             final_points = points
#             trade_book.append({'Entry Time': entry_time, 'Trade Type': trade_type, 'Entry Price': entry_price,
#                                'Stoploss In Points': entry_price - stoploss_long if trade_type == 'LONG' else stoploss_short - entry_price,
#                                'Exit Time': df.iloc[i]['datetime'], 'Exit Price': exit_price,
#                                'SL or Target': 'Target' if points > 0 else 'SL',
#                                'Points Captured': final_points, 'After Costs': (points-3), 'PnL': (points-3)*100, 'ROI%': (points-3)*100/2500})
            
#     # Convert the list of trades into a DataFrame
#     trade_book_df = pd.DataFrame(trade_book)
#     return trade_book_df

# positional_tradebook = execute_trades_positional(atr_df)
# positional_tradebook['year'] = positional_tradebook['Entry Time'].dt.year

In [26]:
# positional_tradebook

In [27]:
# pnl = positional_tradebook['After Costs'].sum()
# pnl

In [28]:
# stats_df = pd.DataFrame(index=range(2017, 2024), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])

# # Iterate over each year
# for year in range(2017, 2024):
#     # Filter trades for the current year
#     year_trades = positional_tradebook[(positional_tradebook['year'] == year)]
    
#     # Calculate total ROI
#     total_roi = year_trades['ROI%'].sum()
    
#     # Calculate total number of trades
#     total_trades = len(year_trades)
    
#     # Calculate win rate
#     win_rate = (year_trades['ROI%'] > 0).mean()*100
    
#     # Calculate average profit per trade
#     avg_profit = year_trades[year_trades['ROI%'] > 0]['ROI%'].mean()
    
#     # Calculate average loss per trade
#     avg_loss = year_trades[year_trades['ROI%'] < 0]['ROI%'].mean()
    
#     # Calculate maximum drawdown
#     max_drawdown = (year_trades['ROI%'].cumsum() - year_trades['ROI%'].cumsum().cummax()).min()
    
#     # Calculate ROI/DD ratio
#     roi_dd_ratio = total_roi / abs(max_drawdown)
    
#     # Store the statistics in the DataFrame
#     stats_df.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# # Calculate overall statistics
# overall_total_roi = stats_df['Total ROI'].sum()
# overall_total_trades = stats_df['Total Trades'].sum()
# overall_win_rate = (positional_tradebook['ROI%'] > 0).mean() * 100
# overall_avg_profit = positional_tradebook[positional_tradebook['ROI%'] > 0]['ROI%'].mean()
# overall_avg_loss = positional_tradebook[positional_tradebook['ROI%'] < 0]['ROI%'].mean()
# overall_max_drawdown = (positional_tradebook['ROI%'].cumsum() - positional_tradebook['ROI%'].cumsum().cummax()).min()
# overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# # Store the overall statistics in the DataFrame
# stats_df.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
# stats_df

In [29]:
stats_df2 = pd.DataFrame(index=range(2017, 2024), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])

# Iterate over each year
for year in range(2017, 2024):
    # Filter trades for the current year
    year_trades = tradebook_w_trail[(tradebook_w_trail['year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['ROI%'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['ROI%'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['ROI%'] > 0]['ROI%'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['ROI%'] < 0]['ROI%'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['ROI%'].cumsum() - year_trades['ROI%'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    
    # Store the statistics in the DataFrame
    stats_df2.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df2['Total ROI'].sum()
overall_total_trades = stats_df2['Total Trades'].sum()
overall_win_rate = (tradebook_w_trail['ROI%'] > 0).mean() * 100
overall_avg_profit = tradebook_w_trail[tradebook_w_trail['ROI%'] > 0]['ROI%'].mean()
overall_avg_loss = tradebook_w_trail[tradebook_w_trail['ROI%'] < 0]['ROI%'].mean()
overall_max_drawdown = (tradebook_w_trail['ROI%'].cumsum() - tradebook_w_trail['ROI%'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df2.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]

In [30]:
# print('Without Trailing, Exit on Target, SL, ATR retracement')
# stats_df

In [31]:
print('Intraday With MA Trailing, Exit on Target, SL, Trailing SL')
print('#10, 20, 60, 14, 4R')
stats_df2

Intraday With MA Trailing, Exit on Target, SL, Trailing SL
#10, 20, 60, 14, 4R


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,-1.8,160,43.75,0.105,-0.115823,-2.685,-0.670391
2018,-2.4,187,33.15508,0.231532,-0.140798,-4.695,-0.511182
2019,-3.39,192,40.104167,0.15039,-0.133661,-5.115,-0.662757
2020,3.975,172,43.023256,0.28723,-0.185806,-4.26,0.933099
2021,-1.755,205,38.536585,0.307785,-0.220932,-6.945,-0.2527
2022,9.735,206,49.029126,0.505842,-0.393857,-8.445,1.152753
2023,5.34,201,49.751244,0.28755,-0.236515,-4.245,1.257951
Overall,9.705,1323.0,42.5548,0.281883,-0.20551,-11.715,0.828425


In [32]:
print('ROIs : ')
pos_roi = positional_tradebook['ROI%'].sum()
trl_roi = tradebook_w_trail['ROI%'].sum()
print('Positional : ', pos_roi, '%')
print('Trailing : ', trl_roi, '%')
print(' ')
print('Points Captured Post Slippage : ')
pos_roi = positional_tradebook['After Costs'].sum()
trl_roi = tradebook_w_trail['After Costs'].sum()
print('Positional : ', pos_roi)
print('Trailing : ', trl_roi)

ROIs : 


NameError: name 'positional_tradebook' is not defined